# Trying to use pocketsphinx to word align

> "Because timing accuracy in ASR is getting progressively worse, look backwards"

- branch: master
- comments: false
- categories: [pocketsphinx, hsi, alignment]

In [1]:
from pydub import AudioSegment
import pocketsphinx
import tempfile

In [2]:
MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
ɾ D
"""

In [3]:
espeak_to_cmudict = {}
for line in MAPPING.split("\n"):
    if line == "":
        continue
    line = line.strip()
    parts = line.split(" ")

    if len(parts) != 2:
        print(line)
        continue
    k, v = line.split(" ")
    if not k in espeak_to_cmudict:
        espeak_to_cmudict[k] = v
    

In [4]:
import re

cmudict_keys = espeak_to_cmudict.keys()
cmudict_keys = sorted(cmudict_keys, key=len, reverse=True)
espeak_regex = re.compile(rf"({'|'.join(cmudict_keys)})")

def cmudictify(espeak):
    espeak = espeak.replace("ˈ", "").replace("ˌ", "")
    return " ".join([espeak_to_cmudict[x] for x in re.findall(espeak_regex, espeak)])

In [35]:
EGTEXT = "Yeah, that's true. I mean, they are the same size and they are a little bit, but I think I I should go more for something that style."
EGPHON = "/jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ɚ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl./"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_7_0719_210_001_main.wav"
EGSTART = 70.028
EGEND = 75.441

In [ ]:
EGTEXT = "no it's a cheetah, ah, yes it's a cheetah ah and, uh, that one is from, uh, India and, uh, it's really"
EGPHON = "/nˈoʊ ɪts ɐ tʃˈiːt ˈɑːɛ jˈɑs ɪts ɐ tʃˈiːtə ˈɑː ænd ˈɛ ðˈæt wˌʌn ɪz fɹˈɑːm ˈɛ ˈɪndiæ ænd ˈɛ ɪts ɹˈili/"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_5_0718_211_002_main.wav"

EGTEXT2 = "So it was standing somewhere in the middle of the room and it was flying around."
EGSTART2 = 122.709
EGEND2 = 126.344

In [7]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [cmudictify(normphon(x)) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output
    

In [8]:
def make_ps_dict(entries):
    counts = {}
    output = []
    lex = sorted(entries)
    for entry in lex:
        count = 1
        if not entry[0] in counts:
            counts[entry[0]] = 1
        else:
            counts[entry[0]] += 1
            count = counts[entry[0]]
        if count != 1:
            subscript = f"({count})"
        else:
            subscript = ""
        output.append(f"{entry[0]}{subscript} {entry[1]}")
    return output

In [9]:
def make_fsg_transitions_from_text(text):
    words = [normword(x) for x in text.split(" ")]
    enum = [x for x in enumerate(words)]
    trans = [(x[0], x[0] + 1, 1.0, x[1]) for x in enum]
    return trans

In [10]:
audio = AudioSegment.from_file(EGFILE)
audio = audio.set_frame_rate(16000)

In [36]:
def align_with_pocketsphinx(text, phones, audio, start, end):
    lex = make_lexicon(text, phones)
    entries = make_ps_dict(lex)
    segments = []

    with (
        tempfile.NamedTemporaryFile(suffix=".dict", delete=False) as dictf,
        tempfile.NamedTemporaryFile(suffix=".raw", delete=False) as wavf,
    ):

        with open(dictf.name, "w") as dictout:
            for entry in entries:
                print(entry)
                dictout.write(entry + "\n")

        audioseg = audio[int(start * 1000):int(end * 1000)]
        audioseg.export(wavf.name, format="s16le", parameters=["-ac", "1", "-acodec", "pcm_s16le", "-f", "s16le", "-ar", "16000"])
        print(wavf.name)

        fsgt = make_fsg_transitions_from_text(text)
        start_state = fsgt[0][0]
        end_state = fsgt[-1][1]

        decoder = pocketsphinx.Decoder(lm=None, dict=dictf.name)
        fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)
        fsg.writefile("/tmp/fsm")
        decoder.add_fsg("dummy", fsg)
        decoder.activate_search("dummy")
        decoder.start_utt()
        # decoder.process_raw(seg.get_array_of_samples('B'))
        decoder.process_raw(wavf.read(), full_utt=True)
        decoder.end_utt()
        for seg in decoder.seg():
            segments.append({
                "word": seg.word,
                "start_frame": seg.start_frame,
                "end_frame": seg.end_frame
            })
    return segments

In [37]:
segs = align_with_pocketsphinx(EGTEXT, EGPHON, audio, EGSTART, EGEND)

a AH
and AH N
are ER
bit B IH D
but B AH T
for F ER
go G OW
i AH
i(2) AY
little L IH D AH
mean M IY N
more M AO R
same S EY M
should SH UH
size S AY Z
something S AH M TH IH NG
style S T AY L
that DH AE T
that's DH AE S
the DH AH
they DH EY
think TH IH NG K
true T R UW
yeah Y AE
/var/folders/d9/cbkhg23x349_t692zq6yhcv00000gn/T/tmpu2pmfssj.raw


ERROR: "fsg_search.c", line 944: Final result does not match the grammar in frame 540


TypeError: 'NoneType' object is not iterable

In [43]:
!/opt/homebrew/bin/sox /tmp/clip.wav $(/opt/homebrew/bin/pocketsphinx soxflags) > /tmp/ps.raw